# Problem Statement

Despite the availability of Malaysia House Rental listings on platforms like iBilik, propertyguru or mudah.my, prospective tenants often struggle to determine whether **rental prices are justified** based on various property attributes.

This project aims to **analyze factors influencing rental prices** to provide potential renters and/or investors with a clearer understanding of the housing market dynamics in this area, and to **predict the expected rental pricing** for the property with given set of attributes.

# Data Collection

The data source for Regression Task is obtained from [Kaggle](https://www.kaggle.com/datasets/ariewijaya/rent-pricing-kuala-lumpur-malaysi/data), where the dataset itself is sourced from web scraping on the mudah.my website.

In [ ]:
# import os
# 
# repo_path = 'MalaysiaRentalPrediction'
# repo_url = 'https://github.com/Xytrix1103/MalaysiaRentalPrediction.git'
# 
# if not os.path.exists(repo_path):
# 	!git clone {repo_url}
# else:
# 	os.chdir(repo_path)
# 	!git pull
# 	os.chdir('..')

# Data Cleaning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Constant for random state seed
RANDOM_STATE_SEED = 1

%matplotlib inline

df = pd.read_csv('./MalaysiaRentalPrediction/mudah-apartment-kl-selangor.csv')
df

We will start by looking at the datatypes of each column

In [ ]:
df.info()

There are some columns that are supposed to be float instead of string. How about the possibility of outliers

In [ ]:
df.describe()

 It's interesting to have rental unit not completed but listed on the website, i.e. the one with `completion_year` after 2024. Next, let's look at the null values

In [ ]:
df.isnull().sum()

From the above info, we can list out the task we need to do:

1. Columns `monthly_rent`, `size` to be converted to type `int`
2. Drop `ads_id` and `prop_name` since they don't help in price prediction
3. Dealing with null values:
    * `completion_year` to use median / mean imputation
    * drop rows where `monthly_rent` is null
    * `rooms` to explore, minimum 1 else it shouldn't be listed
    * `parking` to fill null with 0
    * `bathroom` and `furnished` to be explored
    * `facilities` and `additional_facilities` both be dropped since there are too many null values
4. Dealing with outliers
    * Remove rows where `completion_year` is 2025
    * Remove further outliers


## Drop columns and rows

In [ ]:
df = df.drop(['ads_id', 'prop_name', 'facilities', 'additional_facilities'], axis=1)
df = df.dropna(subset=['monthly_rent'], axis=0)
df

## Datatype Conversion

In [ ]:
df['monthly_rent'] = df['monthly_rent'].str.replace(' ', '').str.extract(r'(\d+\s*\d*)').astype(int)
df['size'] = df['size'].str.replace(' ', '').str.extract(r'(\d+\s*\d*)').astype(int)
df

## Null Imputation

### `completion_year` to use median / mean imputation

We will decide based on the distribution of the values in the column

In [ ]:
sns.histplot(df['completion_year'], kde=True)

Since the graph is skewed, we will use median imputation

In [ ]:
df['completion_year'] = df['completion_year'].fillna(df['completion_year'].median()).astype(int)
df

### `rooms` to explore, minimum 1 else it shouldn't be listed

We look at the rows where `rooms` is null first

In [ ]:
df[df['rooms'].isnull()]

As a Malaysian, since the rental is quite low thus we can assume that `rooms` is 1.

Also, since these rows are the ones where `bathroom` is also null, thus we assume that this 1 room will have 0 bathroom.

In [ ]:
df.loc[df['rooms'].isnull(), ['rooms', 'bathroom']] = [1, 0]
df

### `parking` to fill null with 0

In [ ]:
df.loc[df['parking'].isnull(), 'parking'] = 0
df

Let's check if we can safely convert the above null-imputed columns to integer

In [ ]:
for col in ['rooms', 'parking', 'bathroom']:
    print(df[col].value_counts())

Only the `rooms` column has non-numeric values `'More than 10'` and only 2 occurrences, so we will just set them as 11

In [ ]:
df.loc[df['rooms'] == 'More than 10', 'rooms'] = 11
df

Now we convert them into `int` type

In [ ]:
df['rooms'] = df['rooms'].astype(float).astype(int)
df['parking'] = df['parking'].astype(float).astype(int)
df['bathroom'] = df['bathroom'].astype(float).astype(int)
df

### `furnished` to be explored

We will look at what our `furnished` column look like

In [ ]:
df['furnished'].value_counts()

Let's look at the rows where `furnished` is null

In [ ]:
df[df['furnished'].isnull()]

We explore the typical `monthly_rent` for 1 room for each `furnished` type

In [ ]:
df[df['rooms'] == 1]['monthly_rent'].groupby(df['furnished']).mean()

Okay this looks illogical for partial furnished to be significantly higher than fully furnished, outliers exist so let's use median

In [ ]:
df[df['rooms'] == 1]['monthly_rent'].groupby(df['furnished']).median()

Since all the rooms with `furnished` as null have rental below 800, we just set them to unfurnished

In [ ]:
df.loc[df['furnished'].isnull(), 'furnished'] = 'Not Furnished'
df

### Recheck Null Imputation

We check if all null values are handled

In [ ]:
df.isnull().sum()

### Dealing with Outliers

We first remove the rows where the `completion_year` is after 2024.

In [ ]:
df = df[df['completion_year'] <= 2024]
df

Look at the statistics of the data again

In [ ]:
df.describe()

Using the statistics, we can remove the outliers for `monthly_rent`, `rooms`, `parking`, `bathroom` and `size`

In [ ]:
col_to_rm_outliers = ['monthly_rent', 'rooms', 'parking', 'bathroom', 'size']

def remove_outliers(df, col):

    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

for col in col_to_rm_outliers:
    df = remove_outliers(df, col)

df

Check the statistics again

In [ ]:
df.describe()

Looks more normal now

## EDA



Let's explore the `location` column

In [ ]:
df['location'].value_counts()

See how many states we have

In [ ]:
df['location'].str.split(' - ').str[0].value_counts()

Looks like we only have Selangor and Kuala Lumpur, which is the `region`, how about the cities

In [ ]:
df['location'].str.split(' - ').str[1].value_counts()

Since we have 116 cities here, let's look at the top 15 cities with most number of rental units

In [ ]:
df['location'] = df['location'].str.split(' - ').str[1]
top_cities = df['location'].value_counts().reset_index()[:15]

# Plot bar chart
plt.title('Top 15 Cities with Most Rental Units')
sns.barplot(data=top_cities, x='count', y='location')

Let's see how property types affect the rental price

In [ ]:
type_price = df.groupby(['property_type'])['monthly_rent'].mean().sort_values(ascending=False).reset_index()

# Plot bar chart
plt.title('Average Monthly Rent for each Property Type')
sns.barplot(data=type_price, x='monthly_rent', y='property_type')

As expected, high rise buildings' units are more expensive.

Let's look at the room per price for each city

In [ ]:
df2 = df.copy()
df2['price_per_room'] = df2['monthly_rent'] / df2['rooms']

# Look at only the top 15 most expensive ones
expensive_cities = df2.groupby(['location', 'region'])[['price_per_room']].median().sort_values(by='price_per_room', ascending=False)[:15]

# Plot bar chart
plt.title('Top 15 Cities with Most Expensive Price per Room')
sns.barplot(data=expensive_cities, x='price_per_room', y='location', hue='region')

We can see that these cities consists of quite balance amount of representatives from KL and Selangor, but KL still has a slightly higher pricing than Selangor

Let's see if the completion date of the unit affects the current rental pricing

In [ ]:
pricing_trend = df2.groupby(['completion_year', 'region'])[['price_per_room']].median()

# Plot line graph
plt.title('Price per Room against Completion Year')
sns.lineplot(data=pricing_trend, x='completion_year', y='price_per_room', hue='region')

The completion date slightly affects the rental pricing, with Selangor's being more stable and KL's showing a visible increase in pricing, indicating newer properties in KL can be rented at higher price

Let's explore about the size of rental units

In [ ]:
# Look at only the top 15 most expensive ones
expensive_cities = df2.groupby(['location', 'region'])[['size']].median().sort_values(by='size', ascending=False)[:15]

# Plot bar chart
plt.title('Top 15 Cities with Biggest Rental Units')
sns.barplot(data=expensive_cities, x='size', y='location', hue='region')

It is quite average from the bar chart above, that neither of KL rental units nor Selangor rental units are bigger generally.

Let's also look at the furnished status for KL and Selangor

In [ ]:
df.groupby(['region', 'furnished']).size().unstack(fill_value=0).plot(kind='bar')

From the chart, there are usually less unfurnished rental units listed and more of the partially or fully furnished rental ones.

KL has more furnished rooms than Selangor, whereas Selangor has more unfurnished ones than KL

How about number of rooms?

In [ ]:
df.groupby(['region', 'rooms']).size().unstack(fill_value=0).plot(kind='bar')

Looks like most Malaysian rental units are of type 3 bedrooms

Lastly, let's explore about the property type

In [ ]:
df.groupby(['region', 'property_type']).size().unstack(fill_value=0).plot(kind='bar')

Seems like rental units are mostly consisted of condominium, service residence and apartments, with KL having the most number of condominium units to be rented.

# Regression

Start off by importing relevant libraries

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Model libraries
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor

## Data Preparation

Since we have numerical and categorical columns in our dataset, we will need to perform encoding on the categorical ones

In [ ]:
# Create mapping from property type to average price sorted
type_price_map = type_price.drop('monthly_rent', axis=1).reset_index().set_index('property_type')['index']

# Use label encoding on property type column
df['property_type'] = df['property_type'].map(type_price_map)

# Use label encoding on furnished column
df.loc[df['furnished'] == 'Not Furnished','furnished'] = 0
df.loc[df['furnished'] == 'Partially Furnished','furnished'] = 1
df.loc[df['furnished'] == 'Fully Furnished','furnished'] = 2

# Use label encoding on region column
df.loc[df['region'] == 'Selangor', 'region'] = 0
df.loc[df['region'] == 'Kuala Lumpur', 'region'] = 1

df

Now, we are left with `location`, since there are 116 unique cities and it will be unwise to use one-hot encoding, and location is semi related to region so we will not use it forward in the model training later on.

In [ ]:
df = df.drop('location', axis=1)
df

Now that we are done with the categorical ones into numerical, we have to perform scaling on the numerical columns.

Since `completion_year` are between 1977 to 2024, we can create a new feature called `year_since_completion` which stores the number of years since the completion year to 2024, effectively making an impact for the column towards the pricing difference

In [ ]:
# "Scaling" completion_year
df['year_since_completion'] = 2024 - df['completion_year']
df = df.drop('completion_year', axis=1)
df

For the `size` column, we need to see the distribution to see if it can be scaled using standard scaler.

In [ ]:
# Check if size data are normally distributed to use standard scaler
sns.histplot(df['size'], kde=True)

Since it is normally distributed, we can use standard scaler to scale the feature values

In [ ]:
# Standard scaler
sscaler = StandardScaler()
df['size'] = sscaler.fit_transform(df[['size']])
df

Check if all columns are numerical types

In [ ]:
df.info()

In [ ]:
df['furnished'] = df['furnished'].astype(int)
df['region'] = df['region'].astype(int)

df.info()

## Create feature and target

In [ ]:
X = df.drop('monthly_rent', axis=1)
y = df['monthly_rent']

# Split data to 70% training, 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE_SEED)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

## Model 1: Linear Model

Here we use Ridge Model instead of Linear Regression Model as there are tunable parameters for Ridge model.

In [ ]:
linear_model = Ridge()
linear_model.fit(X_train, y_train)

y_pred_lm = linear_model.predict(X_test)

print('RMSE:', lm_rmse := np.sqrt(mean_squared_error(y_test, y_pred_lm)))
print('MAE:', lm_mae := mean_absolute_error(y_test, y_pred_lm))

### Tuning Ridge Model

In [ ]:
param_grid = {
    'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_search = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

tuned_linear_model = grid_search.best_estimator_
tuned_linear_model.fit(X_train, y_train)

y_pred_tuned_lm = tuned_linear_model.predict(X_test)

print(f'Best parameters: {grid_search.best_params_}')
print('RMSE:', tuned_lm_rmse := np.sqrt(mean_squared_error(y_test, y_pred_tuned_lm)))
print('MAE:', tuned_lm_mae := mean_absolute_error(y_test, y_pred_tuned_lm))

## Model 2: Ensemble Model

Here we are using XGBoost Regressor model as our ensemble model to compare the prediction accuracy with the basic linear model

In [ ]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print('RMSE:', xgb_rmse := np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print('MAE:', xgb_mae := mean_absolute_error(y_test, y_pred_xgb))

### Tuning XGBoost Regressor

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

tuned_xgb_model = grid_search.best_estimator_
tuned_xgb_model.fit(X_train, y_train)

y_pred_tuned_xgb = tuned_xgb_model.predict(X_test)

print(f'Best parameters: {grid_search.best_params_}')
print('RMSE:', tuned_xgb_rmse := np.sqrt(mean_squared_error(y_test, y_pred_tuned_xgb)))
print('MAE:', tuned_xgb_mae := mean_absolute_error(y_test, y_pred_tuned_xgb))

# Model Results

Finally, we look at the model results

In [ ]:
pd.DataFrame({
    'Model': ['Ridge Model', 'Tuned Ridge Model', 'XGBoost Regressor', 'Tuned XGBoost Regressor'],
    'RMSE': [lm_rmse, tuned_lm_rmse, xgb_rmse, tuned_xgb_rmse],
    'MAE': [lm_mae, tuned_lm_mae, xgb_mae, tuned_xgb_mae]
})

In general, ensemble learning model performs better than basic linear model, and it is no difference in this dataset.

Tuning does improve the model's performance, but the difference is not very significant in both models.

As conclusion. the **tuned XGBoost Regressor Model** gives the lowest RMSE and MAE, thus this is the best model.